### カリキュラム - 【AIコース】 Tuning2 Hyper-Parameters

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam

Using TensorFlow backend.
/Users/yasu/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Loading data
seed = 7
np.random.seed(seed)

url = "http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
df_iris = pd.read_csv(url, header = None)
df_iris.columns = [
    "sepal length in cm",
    "sepal width in cm",
    "petal length in cm",
    "petal width in cm",
    "class"
]

In [3]:
# Preprocessing
# split into input X and label y
X = df_iris.values[:,0:4].astype(float)
y = df_iris.values[:,4]
# Convert string to integer
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
# Convert integer to one-hot
y_dummy = np_utils.to_categorical(y_encoded)
X_train, X_test, y_train, y_test = train_test_split(X, y_dummy)

In [4]:
# Optimize hyperparameters - Search range
prm_grid = {
    "n_layer":    range(1, 4 + 1),
    "opt":        ["SGD", "RMSprop", "Adagrad", "Adadelta", "Adam"],
    "lr":         [10 ** (-i) for i in range(1, 4)],
    "batch_size": range(10, 100, 10),
}

prm_grid_layer = {
    "n_unit":     range(1, 10 + 1),
    "activation": ["sigmoid", "tanh", "relu"],
}

dict_opt = {
    "SGD":      lambda lr: SGD(lr = lr),
    "RMSprop":  lambda lr: RMSprop(lr = lr),
    "Adagrad":  lambda lr: Adagrad(lr = lr),
    "Adadelta": lambda lr: Adadelta(lr = lr),
    "Adam":     lambda lr: Adam(lr = lr),
}

In [5]:
# Model selection
# Define model
def train(X, y, prm, n_epoch):
    layers = prm["layers"]
    optimizer = dict_opt[prm["opt"]](lr = prm["lr"])
    batch_size = prm["batch_size"]
    
    # Define model architecture
    model = Sequential()
    for i, l in enumerate(layers):
        if i == 0:
            model.add(Dense(l["n_unit"], input_dim = 4, activation = l["activation"]))
        else:
            model.add(Dense(l["n_unit"], activation = l["activation"]))
    model.add(Dense(3, activation = "softmax"))
    
    # Compile model
    model.compile(
        loss =      "categorical_crossentropy",
        optimizer = optimizer,
        metrics =   ["accuracy"],
    )
    validation_split = 0.3
    history = model.fit(X, y, 
                        epochs =           n_epoch,
                        batch_size =       batch_size,
                        validation_split = validation_split,
                        verbose =           False
                       )
    return model, history

In [6]:
# 評価関数の定義
def mean_best_10_val_acc(history):
    df = pd.DataFrame(history.history)
    # sort_valuesは昇順のため[::-1]で降順にする
    return df["val_acc"].sort_values()[::-1][:10].mean()

In [7]:
# Run train function
from sklearn.model_selection import ParameterSampler

n_sample = 10
n_epoch = 200

prms = ParameterSampler(prm_grid, n_iter = n_sample)
best_score = 0
best_model = None
results = []
for i, prm in enumerate(prms):
    layers = list(ParameterSampler(prm_grid_layer, n_iter = prm["n_layer"]))
    prm["layers"] = layers
    print("sample{}: ".format(i + 1), prm)
    # Learnig
    model, history = train(X_train, y_train, prm, n_epoch)
    prm["score"] = mean_best_10_val_acc(history)
    print(prm["score"])
    results.append(prm)
    # Update score with best score
    if best_score < prm["score"]:
        best_score = prm["score"]
        best_model = model
df_results = pd.DataFrame(results)

sample1:  {'opt': 'SGD', 'n_layer': 2, 'lr': 0.001, 'batch_size': 60, 'layers': [{'n_unit': 10, 'activation': 'relu'}, {'n_unit': 3, 'activation': 'relu'}]}
0.29411765933036804
sample2:  {'opt': 'RMSprop', 'n_layer': 4, 'lr': 0.1, 'batch_size': 40, 'layers': [{'n_unit': 9, 'activation': 'tanh'}, {'n_unit': 7, 'activation': 'sigmoid'}, {'n_unit': 1, 'activation': 'relu'}, {'n_unit': 7, 'activation': 'relu'}]}
0.29411765933036804
sample3:  {'opt': 'Adagrad', 'n_layer': 3, 'lr': 0.001, 'batch_size': 60, 'layers': [{'n_unit': 6, 'activation': 'relu'}, {'n_unit': 2, 'activation': 'tanh'}, {'n_unit': 10, 'activation': 'sigmoid'}]}
0.29411765933036804
sample4:  {'opt': 'RMSprop', 'n_layer': 2, 'lr': 0.01, 'batch_size': 30, 'layers': [{'n_unit': 6, 'activation': 'relu'}, {'n_unit': 3, 'activation': 'tanh'}]}
0.9735293896759257
sample5:  {'opt': 'Adagrad', 'n_layer': 2, 'lr': 0.001, 'batch_size': 70, 'layers': [{'n_unit': 9, 'activation': 'tanh'}, {'n_unit': 1, 'activation': 'sigmoid'}]}
0.2941

In [8]:
# DataFrameの文字表示の制限を10000文字までにする
pd.set_option("display.max_colwidth", 10000)
# scoreを降順にする
df_results.sort_values("score")[::-1]

,batch_size,layers,lr,n_layer,opt,score
3,30,"[{'n_unit': 6, 'activation': 'relu'}, {'n_unit': 3, 'activation': 'tanh'}]",0.010,2,RMSprop,0.973529
5,80,"[{'n_unit': 7, 'activation': 'tanh'}]",0.100,1,Adam,0.970588
7,90,"[{'n_unit': 5, 'activation': 'relu'}]",0.001,1,SGD,0.500000
9,10,"[{'n_unit': 3, 'activation': 'sigmoid'}, {'n_unit': 4, 'activation': 'relu'}, {'n_unit': 3, 'activation': 'tanh'}, {'n_unit': 5, 'activation': 'tanh'}]",0.100,4,Adam,0.341176
8,50,"[{'n_unit': 6, 'activation': 'relu'}, {'n_unit': 3, 'activation': 'sigmoid'}, {'n_unit': 7, 'activation': 'tanh'}]",0.010,3,SGD,0.294118
6,40,"[{'n_unit': 2, 'activation': 'relu'}, {'n_unit': 10, 'activation': 'sigmoid'}, {'n_unit': 6, 'activation': 'relu'}]",0.010,3,Adadelta,0.294118
4,70,"[{'n_unit': 9, 'activation': 'tanh'}, {'n_unit': 1, 'activation': 'sigmoid'}]",0.001,2,Adagrad,0.294118
2,60,"[{'n_unit': 6, 'activation': 'relu'}, {'n_unit': 2, 'activation': 'tanh'}, {'n_unit': 10, 'activation': 'sigmoid'}]",0.001,3,Adagrad,0.294118
1,40,"[{'n_unit': 9, 'activation': 'tanh'}, {'n_unit': 7, 'activation': 'sigmoid'}, {'n_unit': 1, 'activation': 'relu'}, {'n_unit': 7, 'activation': 'relu'}]",0.100,4,RMSprop,0.294118
0,60,"[{'n_unit': 10, 'activation': 'relu'}, {'n_unit': 3, 'activation': 'relu'}]",0.001,2,SGD,0.294118


In [9]:
scores = best_model.evaluate(X_test, y_test, verbose = 0)
pd.Series(scores, index = model.metrics_names)

loss    0.111261
acc     0.973684
dtype: float64